<a href="https://colab.research.google.com/github/JulianMeigen/ML-handson/blob/main/notebooks/7.0-SNJMMH-Day7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment Day 7

## Team members:
- Samuel Nebgen s6sanebg@uni-bonn.de
- Muhammad Humza Arain s27marai@uni-bonn.de
- Julian Meigen s82jmeig@uni-bonn.de

## 16.09.2025

Contributions were made by all team members in around the same amount, either based on discussions or coding.

In [2]:
!gdown --folder https://drive.google.com/drive/folders/1VESm-JaHEqPJmM23iLW1mEJsuI2mLBdx?usp=sharing

Retrieving folder contents
Processing file 1i6W9fI3sGEn6V9xBlt2MxlonZXOTLZjg load-subgraph_doc.ipynb
Processing file 1qZpQzFMRzuYQ0xoQJcUNe7CRBMS2mRmz subgraph_hop_1.pt
Processing file 1iz_FOBs9k7m9z3lDtRIXzRkd92tL_EK- subgraph.pt
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1i6W9fI3sGEn6V9xBlt2MxlonZXOTLZjg
To: /content/ML-HandsOn/load-subgraph_doc.ipynb
100% 2.92k/2.92k [00:00<00:00, 10.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qZpQzFMRzuYQ0xoQJcUNe7CRBMS2mRmz
To: /content/ML-HandsOn/subgraph_hop_1.pt
100% 10.5M/10.5M [00:00<00:00, 88.9MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1iz_FOBs9k7m9z3lDtRIXzRkd92tL_EK-
From (redirected): https://drive.google.com/uc?id=1iz_FOBs9k7m9z3lDtRIXzRkd92tL_EK-&confirm=t&uuid=28773feb-027d-4abb-a165-8187c93a078a
To: /content/ML-HandsOn/subgraph.pt
100% 1.64G/1.64G [00:13<00:00, 123MB/s]
Download co

In [5]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.5 MB/s eta 0:00:00


In [6]:
import torch
import torch_geometric
import numpy as np
import pandas as pd
import networkx as nx
import plotly
from torch_geometric.utils import to_networkx
from torch.nn import Embedding
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Task 1 Perform a node labeling task with a Graph ML model

## a) Load the graph dataset (ogbn-proteins) into pytorch-geometric

We are directly using a Subgraph

In [9]:
path_big = "/content/ML-HandsOn/subgraph.pt"
path_small = "/content/ML-HandsOn/subgraph_hop_1.pt"

dataset = torch.load(path_big, weights_only=False)

data = dataset["graph"]
print(data)

In [ ]:
G = to_networkx(data, to_undirected=True)
print(G)

## b) Create a train, val, test split on the nodes or load the masks via pytorch-geometric.

### i. Create a subgraph if the computation is too expensive.

In [ ]:
num_nodes = data.num_nodes
perm = torch.randperm(num_nodes)

train_size = int(0.8 * num_nodes)
val_size = int(0.1 * num_nodes)

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[perm[:train_size]] = True
val_mask[perm[train_size:train_size + val_size]] = True
test_mask[perm[train_size + val_size:]] = True

data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

print(data.y)

In [ ]:
print(len(data.y[data.train_mask]))
print(len(data.y[data.val_mask]))
print(len(data.y[data.test_mask]))

## c) Initialize the graph with random node embeddings.

In [ ]:
number_nodes = data.num_nodes
embedding_dim = 112
node_embeddings = Embedding(number_nodes, embedding_dim)

## d) Define a graph convolutional neural network class with two layers using pytorch-geometric..

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, num_nodes, embedding_dim, hidden_dim, out_dim):
        super().__init__()
        self.emb = torch.nn.Embedding(num_nodes, embedding_dim)
        self.conv1 = GCNConv(embedding_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, out_dim)

    def forward(self, edge_index):
        # get embeddings for all nodes
        x = self.emb.weight
        # apply GCN layers
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [ ]:
model_gcn = GCN(num_nodes=data.num_nodes, embedding_dim=128, hidden_dim=64, out_dim=121)

### i. Train your model on the train dataset using an optimizer and a loss function for a multilabel classification task for 100 epochs

### ii. Test your model on the test set and evaluate it with accuracy, AUROC, precision, recall and F1 score.

## e) Set up a hyperparameter optimization pipeline with nested 5-fold cross-validation

### i. Familiarize yourself with the hyperparameter optimization package optuna (https://optuna.org/ )

### ii. Integrate the logging package mlflow (https://mlflow.org/) to log your metrics.

### iii. Train and test your models and report the evaluation metrics with mean and std for the nested CV.